#### 큰 흐름
- 라이브러리 로드
- 데이터 확인
- 모델 학습
- 제출 파일

---

##### 라이브러리 로드

In [30]:
# !pip install scikit-learn xgboost

import numpy as np
import pandas as pd
from datetime import datetime, timedelta

df = pd.read_csv("data/train.csv")
df

,신고접수번호,접수경로,신고접수일시,시군구,접수분류,긴급구조종류
0,JGT2IBW4,이동전화,20130101_0001,NaN,안내,NaN
1,74ED11Z4,기타,20130101_0002,NaN,안내,NaN
2,B4I8RIBW,이동전화,20130101_0002,NaN,안내,NaN
3,482FI3AJ,이동전화,20130101_0003,북구,출동,구급
4,AR9N3QT4,이동전화,20130101_0004,NaN,안내,NaN
...,...,...,...,...,...,...
6415797,KAOUMKLU,기타,20220630_2354,NaN,안내,NaN
6415798,LAI1MQ2K,이동전화,20220630_2354,사상구,출동,구급
6415799,N5KF2ELA,이동전화,20220630_2356,동구,출동,구급
6415800,53Q98D6Z,의료지도연결,20220630_2358,NaN,안내,NaN


In [45]:
# print(df["신고접수번호"].value_counts())
# print(df["접수경로"].value_counts())
# print(df["신고접수일시"].value_counts())
# print(df["시군구"].value_counts())
# print(df["접수분류"].value_counts().sum())
print(df["긴급구조종류"].value_counts())

print(df.loc[df["긴급구조종류"].isnull() == False, ["접수분류","긴급구조종류"]].value_counts())

긴급구조종류
구급    1582608
기타     403642
구조     321461
화재      92466
Name: count, dtype: int64
접수분류  긴급구조종류
출동    구급        1582605
      기타         403474
      구조         321461
      화재          92330
안내    기타            168
      화재            136
      구급              3
Name: count, dtype: int64
           신고접수번호    접수경로         신고접수일시  시군구 접수분류 긴급구조종류
0        JGT2IBW4    이동전화  20130101_0001  NaN   안내    NaN
1        74ED11Z4      기타  20130101_0002  NaN   안내    NaN
2        B4I8RIBW    이동전화  20130101_0002  NaN   안내    NaN
3        482FI3AJ    이동전화  20130101_0003   북구   출동     구급
4        AR9N3QT4    이동전화  20130101_0004  NaN   안내    NaN
...           ...     ...            ...  ...  ...    ...
6415797  KAOUMKLU      기타  20220630_2354  NaN   안내    NaN
6415798  LAI1MQ2K    이동전화  20220630_2354  사상구   출동     구급
6415799  N5KF2ELA    이동전화  20220630_2356   동구   출동     구급
6415800  53Q98D6Z  의료지도연결  20220630_2358  NaN   안내    NaN
6415801  EG6XYE3F    이동전화  20220630_2359   동구   출동     구급

[641580

In [4]:
df["dt"] = df["신고접수일시"].apply(lambda x : datetime.strptime(x, "%Y%m%d_%H%M"))
df["_dt"] = df["dt"] + timedelta(hours=6)
df["_date"] = df._dt.dt.date.astype(str)
df["_hour"] = df._dt.dt.hour
df

,신고접수번호,접수경로,신고접수일시,시군구,접수분류,긴급구조종류,dt,_dt,_date,_hour
0,JGT2IBW4,이동전화,20130101_0001,NaN,안내,NaN,2013-01-01 00:01:00,2013-01-01 06:01:00,2013-01-01,6
1,74ED11Z4,기타,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
2,B4I8RIBW,이동전화,20130101_0002,NaN,안내,NaN,2013-01-01 00:02:00,2013-01-01 06:02:00,2013-01-01,6
3,482FI3AJ,이동전화,20130101_0003,북구,출동,구급,2013-01-01 00:03:00,2013-01-01 06:03:00,2013-01-01,6
4,AR9N3QT4,이동전화,20130101_0004,NaN,안내,NaN,2013-01-01 00:04:00,2013-01-01 06:04:00,2013-01-01,6
...,...,...,...,...,...,...,...,...,...,...
6415797,KAOUMKLU,기타,20220630_2354,NaN,안내,NaN,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415798,LAI1MQ2K,이동전화,20220630_2354,사상구,출동,구급,2022-06-30 23:54:00,2022-07-01 05:54:00,2022-07-01,5
6415799,N5KF2ELA,이동전화,20220630_2356,동구,출동,구급,2022-06-30 23:56:00,2022-07-01 05:56:00,2022-07-01,5
6415800,53Q98D6Z,의료지도연결,20220630_2358,NaN,안내,NaN,2022-06-30 23:58:00,2022-07-01 05:58:00,2022-07-01,5


In [17]:
df_ = df.pivot_table(index = ['_date', '_hour'], columns = '접수분류', aggfunc='size').reset_index()
print(df_)

접수분류        _date  _hour    안내    출동
0      2013-01-01      6  47.0  30.0
1      2013-01-01      7  55.0  26.0
2      2013-01-01      8  40.0  32.0
3      2013-01-01      9  24.0  21.0
4      2013-01-01     10  28.0  19.0
...           ...    ...   ...   ...
83227  2022-07-01      1  78.0  58.0
83228  2022-07-01      2  85.0  43.0
83229  2022-07-01      3  57.0  33.0
83230  2022-07-01      4  53.0  47.0
83231  2022-07-01      5  63.0  41.0

[83232 rows x 4 columns]


In [20]:
# 야간 안내, 출동
night_date = df_[(df_["_hour"] >= 0) & (df_["_hour"] < 15)].groupby("_date").sum()
night_date = night_date.reset_index()
night_date.drop(["_hour"], axis=1, inplace=True)
night_date.columns = ["date", "night_0", "night_1"]
# 주간 안내, 출동
day_date = df_[(df_["_hour"] >= 15) & (df_["_hour"] < 24)].groupby("_date").sum()
day_date = day_date.reset_index()
day_date.drop(["_hour"], axis=1, inplace=True)
day_date.columns = ["date", "day_0", "day_1"]

# print(night_date)
# print(day_date)

print("\n==============\n")
# 야간 신고 건수(날짜별)
night_date_all = df_[(df_["_hour"] >= 0) & (df_["_hour"] < 15)].groupby("_date").sum()
night_date_all["night_y"] = night_date_all["안내"] + night_date_all["출동"]
night_date_all = night_date_all.reset_index()
night_date_all.drop(["_hour", "안내", "출동"], axis=1, inplace=True)
night_date_all.columns = ["date", "night_y"]
# 주간 신고 건수(날짜별)
day_date_all = df_[(df_["_hour"] >= 15) & (df_["_hour"] < 24)].groupby("_date").sum()
day_date_all["day_y"] = day_date_all["안내"] + day_date_all["출동"]
day_date_all = day_date_all.reset_index()
day_date_all.drop(["_hour", "안내", "출동"], axis=1, inplace=True)
day_date_all.columns = ["date", "day_y"]
print(night_date_all)
# print(day_date_all)



            date  night_y
0     2013-01-01    613.0
1     2013-01-02    785.0
2     2013-01-03   1013.0
3     2013-01-04   1005.0
4     2013-01-05    791.0
...          ...      ...
3464  2022-06-27   1375.0
3465  2022-06-28   1030.0
3466  2022-06-29   1159.0
3467  2022-06-30   1058.0
3468  2022-07-01    708.0

[3469 rows x 2 columns]


In [21]:
data = pd.concat([night_date, day_date], axis="columns")
data = pd.merge(left=night_date, right=day_date, on="date")
data

,date,night_0,night_1,day_0,day_1
0,2013-01-01,406.0,207.0,1851.0,206.0
1,2013-01-02,491.0,294.0,289.0,272.0
2,2013-01-03,766.0,247.0,316.0,261.0
3,2013-01-04,781.0,224.0,328.0,294.0
4,2013-01-05,467.0,324.0,492.0,294.0
...,...,...,...,...,...
3463,2022-06-26,865.0,491.0,670.0,359.0
3464,2022-06-27,782.0,593.0,682.0,417.0
3465,2022-06-28,608.0,422.0,499.0,385.0
3466,2022-06-29,684.0,475.0,471.0,356.0


In [28]:
data_all = pd.concat([night_date_all, day_date_all], axis="columns")
data_all = pd.merge(left=night_date_all, right=day_date_all, on="date")
data_all

,date,night_y,day_y
0,2013-01-01,613.0,2057.0
1,2013-01-02,785.0,561.0
2,2013-01-03,1013.0,577.0
3,2013-01-04,1005.0,622.0
4,2013-01-05,791.0,786.0
...,...,...,...
3463,2022-06-26,1356.0,1029.0
3464,2022-06-27,1375.0,1099.0
3465,2022-06-28,1030.0,884.0
3466,2022-06-29,1159.0,827.0


In [29]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# df_
# night_
# day_

dates = data["date"].to_list()
night_y = data_all["night_y"].to_list()
day_y = data_all["day_y"].to_list()
night_0 = data["night_0"].to_list()
night_1 = data["night_1"].to_list()
day_0 = data["day_0"].to_list()
day_1 = data["day_1"].to_list()

fig = make_subplots(rows=3, cols=2)
fig.update_layout(title_text=f"일별 119 안내 출동")
fig.add_trace(go.Scatter(x=dates, y=night_y, name="야간 신고 건수"), row=1, col=1)
fig.add_trace(go.Scatter(x=dates, y=day_y, name="주간 신고 건수"), row=1, col=2)
fig.add_trace(go.Scatter(x=dates, y=night_0, name="night_안내"), row=2, col=1)
fig.add_trace(go.Scatter(x=dates, y=night_1, name="night_출동"), row=3, col=1)
fig.add_trace(go.Scatter(x=dates, y=day_0, name="day_안내"), row=2, col=2)
fig.add_trace(go.Scatter(x=dates, y=day_1, name="day_출동"), row=3, col=2)
fig.show()
